In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [3]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import pathlib
from glob import glob

from scipy.io import loadmat, savemat
import os
import math
import numpy as np
import pandas as pd
import numpy as np
import librosa

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

print(tf.__version__)

2.8.2


In [4]:
# Loading csv file
location = "Audio_Classification-MLSP/"
annot = pd.read_csv(location + "annotations.csv")
mapping = {"Bark": 0, "Meow": 1, "Siren": 2, "Shatter": 3, "Knock": 4, "Crying_and_sobbing": 5, "Microwave_oven": 6, "Vehicle_horn_and_car_horn_and_honking": 7, "Doorbell": 8, "Walk_and_footsteps": 9}

In [5]:
# Loading X and y from archives
l = np.load(location + "Dataset.npz")
X = l['X']
y = l['y']
signal_len = l['signal_len']

In [21]:
# Loading test Dataset
test_loc = "Audio_Classification-MLSP-test/"
files = glob(test_loc + "*")

In [22]:
file_name = []
test_data = []
for f in files:
  test_data.append(np.load(f)[0])
  file_name.append(f[len(test_loc):])

In [24]:
cnn1 = tf.keras.models.load_model('cnn1')

In [32]:
test_data[0].shape

(128, 188)

In [47]:
X = np.zeros([201, 128, 2700])
signal_len = []
for i in range(len(test_data)):
  signal_len.append(test_data[i].shape[-1])
  X[i, :, :test_data[i].shape[-1]] = test_data[i]

In [48]:
def DuplicatePadding(X, signal_length, sample_length):
  # Duplicate pad samples with signal_length less than sample_length
  for i in range(len(X)):
    dp = signal_length[i]
    while dp<sample_length:
      X[i, :, dp:dp+signal_length[i]] = X[i, :, :signal_length[i]]
      signal_length[i] = sample_length
      dp = dp + signal_length[i]
  return X, signal_length

In [49]:
def SlidingWindow(X, y, signal_length, sample_length, hop_size):
  ## Slice long samples into multiple samples
  data = []
  labels = []
  for i in range(len(y)):
    if signal_length[i] == sample_length:
      data.append(X[i, :, :signal_length[i]])
      labels.append(y[i])
    else:
      p = 0
      while p + sample_length<=signal_length[i]:
        data.append(X[i, :, p:p + sample_length])
        labels.append(y[i])
        p = p + hop_size
      data.append(np.concatenate([X[i, :, p:signal_length[i]], X[i, :, :sample_length - (signal_length[i]-p)]], axis = -1))
      labels.append(y[i])
      
  data = np.array(data)
  labels = np.array(labels)
  return data, labels

In [50]:
sample_len = 300
hop_size = 150

In [51]:
X_test, signal_len_test = DuplicatePadding(X, signal_len, sample_len)

In [53]:
X_test_list = []
y_test = y[:201] # Random labels
for x, y in zip(X_test, y_test):
  x, _ = SlidingWindow(np.expand_dims(x, axis = 0), list(y), signal_len_test, sample_len, hop_size)
  del _
  X_test_list.append(x)

In [54]:
pred_list = []
for x in X_test_list:
  pred = cnn1.predict(x)
  pred_list.append(np.mean(pred, axis = 0))
predicted = np.array(pred_list)
y_pred = np.argmax(predicted, axis = 1)
# y_pred = tf.keras.utils.to_categorical(y_pred, num_classes)
# train_acuracy = history.history['acc'][-1]
# test_accuracy = accuracy_score(y_test, y_pred)
# f1score = f1_score(y_test, y_pred, average = 'weighted')
# cm = confusion_matrix(np.argmax(y_test, axis = 1), np.argmax(y_pred, axis = 1))
# print(train_acuracy)
# print(test_accuracy)
# print(f1score)
# print(cm)

In [59]:
y_pred[70]

2

In [60]:
pred_class = []
for p in y_pred:
  c = list(mapping.keys())[list(mapping.values()).index(p)]
  pred_class.append(c)

In [61]:
c

'Crying_and_sobbing'

In [62]:
df = pd.DataFrame({"fileid": file_name, "prediction": c})

In [63]:
df

,fileid,prediction
0,MLSP_test1457299.npy,Crying_and_sobbing
1,MLSP_test208923.npy,Crying_and_sobbing
2,MLSP_test1451403.npy,Crying_and_sobbing
3,MLSP_test1443477.npy,Crying_and_sobbing
4,MLSP_test1379148.npy,Crying_and_sobbing
...,...,...
196,MLSP_test80263.npy,Crying_and_sobbing
197,MLSP_test7835.npy,Crying_and_sobbing
198,MLSP_test7872.npy,Crying_and_sobbing
199,MLSP_test77893.npy,Crying_and_sobbing


In [64]:
df.to_csv('predictions.csv')